# steps

- h3 sort in normal (ascending order)
- this puts hexes in order like `[child_0, child_1, child_2, ..., child_6, parent]`
- then zip through list **backwards** to remove children and duplicates, zero these guys out
- then we use a stack moving up the list **forwards** to find cells we can compact

In [1]:
import numpy as np
import h3

In [2]:
h = h3.geo_to_h3(0,0,5)

In [3]:
h

'85754e67fffffff'

In [4]:
h3.string_to_h3(h)

601042424243945471

In [5]:
def is_first_child(h):
    if h3.h3_get_resolution(h) == 0:
        # res zero cells go right through
        return False
    
    p = h3.h3_to_parent(h)
    children = h3.h3_to_children(p)
    first_child = min(children)
    
    return h == first_child

def rich_compare(a, b):
    if a == b:
        return 0
    
    res_a = h3.h3_get_resolution(a)
    res_b = h3.h3_get_resolution(b)
    
    # need child < parent

    # if b is parent of a
    if res_a > res_b:
        if h3.h3_to_parent(a, res_b) == b:
            return -1

    # if a is parent of b
    if res_a < res_b:
        if h3.h3_to_parent(b, res_a) == a:
            return +1
    
    mask = 0x800f_ffff_ffff_ffff  # not sure why we have the 8 here..?
#     mask = 0x800f_ffff_ffff_ffff
    
    a = h3.string_to_h3(a)
    b = h3.string_to_h3(b)
    if (a & mask) < (b & mask):
        return -2
    else:
        return +2
    
def simple_compare(a, b): 
    a = h3.string_to_h3(a)
    b = h3.string_to_h3(b)
    mask = 0x800f_ffff_ffff_ffff

    # return np.sign((a & mask) - (b & mask))  # also works
    return (a & mask) - (b & mask)


from functools import cmp_to_key
def h3order(hexes):
    hexes = sorted(hexes, key = cmp_to_key(simple_compare))

    return hexes

In [6]:
# hexes =  list(h3.h3_to_children(a, 2)) + list(h3.h3_to_children(a, 1))

b = list(h3.get_res0_indexes())[0]

hexes =  [b] + list(h3.h3_to_children(b, 1)) + list(h3.h3_to_children(b, 2))

sorted(hexes, key = cmp_to_key(simple_compare)) == sorted(hexes, key = cmp_to_key(rich_compare))

True

In [21]:
get_remaining_siblings(h)

['85754e63fffffff',
 '85754e6bfffffff',
 '85754e6ffffffff',
 '85754e73fffffff',
 '85754e77fffffff',
 '85754e7bfffffff']

In [23]:
import toolz

ModuleNotFoundError: No module named 'toolz'

In [57]:
# def get_siblings(h):
#     p = h3.h3_to_parent(h)
#     siblings = {s: 0 for s in h3.h3_to_children(p)}
#     siblings[h] = 1
    
#     return siblings

def next_sibling(siblings):
    p = list(siblings)[0]
    
    remaining_siblings = h3.h3_to_children(p) - set(siblings)
    
    if remaining_siblings:
        return min(remaining_siblings)
    else:
        return None
        

In [56]:
next_sibling({h})

'86754e647ffffff'

In [49]:
get_siblings(h)

{'85754e63fffffff': 0,
 '85754e7bfffffff': 0,
 '85754e73fffffff': 0,
 '85754e77fffffff': 0,
 '85754e6bfffffff': 0,
 '85754e6ffffffff': 0,
 '85754e67fffffff': 1}

In [51]:
next_sibling(get_siblings(h))

'85754e63fffffff'

In [22]:
b = list(h3.get_res0_indexes())[0]
hexes = h3.h3_to_children(b, 1)


stack = []

def step(stack, h):   
    out = []
    
    if stack:
        # if h is the next thing we're expecting
        if h == stack[-1][0]:
            stack[-1] = stack[-1][1:]
        else:
            # flush the stack
            for e in stack:
                out += e
            stack[:] = []


    if not stack:
        if is_first_child(h):
            stack.append(get_remaining_siblings(h))
            return []
        else:
            return [h]
    

            
            
    # also need to check if the stack rolls up
            
            
        
    return hexes
    

In [33]:
stack = [1]
a = stack

stack[:] = []

In [34]:
a

[]

In [11]:
compact(hexes)

['81983ffffffffff',
 '81987ffffffffff',
 '8198bffffffffff',
 '8198fffffffffff',
 '81993ffffffffff',
 '81997ffffffffff',
 '8199bffffffffff']